In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from deap import algorithms, base, creator, tools

In [2]:
iris = load_iris()
X, y = iris.data, iris.target

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
np.max(X_train, axis=0)

array([7.7, 4.4, 6.7, 2.5])

In [5]:
np.min(X_train, axis=0)

array([4.3, 2. , 1. , 0.1])

In [6]:
sepal_length = ctrl.Antecedent(np.arange(4.3, 7.8, 0.1), 'sepal_length')
sepal_width = ctrl.Antecedent(np.arange(2.0, 4.5, 0.1), 'sepal_width')
petal_length = ctrl.Antecedent(np.arange(1.0, 6.8, 0.1), 'petal_length')
petal_width = ctrl.Antecedent(np.arange(0.1, 2.6, 0.1), 'petal_width')
y_class = ctrl.Consequent(np.arange(0, 3, 1), 'y_class')

sl_mid = 4.3 + ((7.7 - 4.3)/2)
sepal_length['poor'] = fuzz.trimf(sepal_length.universe, [4.3, 4.3, sl_mid])
sepal_length['average'] = fuzz.trimf(sepal_length.universe, [4.3, sl_mid, 7.7])
sepal_length['good'] = fuzz.trimf(sepal_length.universe, [sl_mid, 7.7, 7.7])

sw_mid = 2.0 + ((4.4 - 2.0)/2)
sepal_width['poor'] = fuzz.trimf(sepal_width.universe, [2.0, 2.0, sw_mid])
sepal_width['average'] = fuzz.trimf(sepal_width.universe, [2.0, sw_mid, 4.4])
sepal_width['good'] = fuzz.trimf(sepal_width.universe, [sw_mid, 4.4, 4.4])

pl_mid = 1.0 + ((6.7 - 1.0)/2)
petal_length['poor'] = fuzz.trimf(petal_length.universe, [1.0, 1.0, pl_mid])
petal_length['average'] = fuzz.trimf(petal_length.universe, [1.0, pl_mid, 6.7])
petal_length['good'] = fuzz.trimf(petal_length.universe, [pl_mid, 6.7, 6.7])

pw_mid = 0.1 + ((2.5 - 0.1)/2)
petal_width['poor'] = fuzz.trimf(petal_width.universe, [0.1, 0.1, pw_mid])
petal_width['average'] = fuzz.trimf(petal_width.universe, [0.1, pw_mid, 2.5])
petal_width['good'] = fuzz.trimf(petal_width.universe, [pw_mid, 2.5, 2.5])

y_class['Iris-setosa'] = fuzz.trimf(y_class.universe, [0, 0, 1])
y_class['Iris-versicolor'] = fuzz.trimf(y_class.universe, [0, 1, 2])
y_class['Iris-virginica'] = fuzz.trimf(y_class.universe, [1, 2, 2])

sepal_length = ctrl.Antecedent(np.arange(x[:, 0].min(), x[:, 0].max(), 0.1), 'sepal_length')
sepal_width = ctrl.Antecedent(np.arange(x[:, 1].min(), x[:, 1].max(), 0.1), 'sepal_width')
petal_length = ctrl.Antecedent(np.arange(x[:, 2].min(), x[:, 2].max(), 0.1), 'petal_length')
petal_width = ctrl.Antecedent(np.arange(x[:, 3].min(), x[:, 3].max(), 0.1), 'petal_width')
y_class = ctrl.Consequent(np.arange(0, 3, 1), 'y_class')

#sepal_length.automf(3)
mid = x[:, 0].min() + ((x[:, 0].max() - x[:, 0].min())/2)
sepal_length['poor'] = fuzz.trimf(sepal_length.universe, [x[:, 0].min(), x[:, 0].min(), mid])
sepal_length['average'] = fuzz.trimf(sepal_length.universe, [x[:, 0].min(), mid, x[:, 0].max()])
sepal_length['good'] = fuzz.trimf(sepal_length.universe, [mid, x[:, 0].max(), x[:, 0].max()])
#sepal_width.automf(3)
sw_mid = x[:, 1].min() + ((x[:, 1].max() - x[:, 1].min())/2)
sepal_width['poor'] = fuzz.trimf(sepal_width.universe, [x[:, 1].min(), x[:, 1].min(), sw_mid])
sepal_width['average'] = fuzz.trimf(sepal_width.universe, [x[:, 1].min(), sw_mid, x[:, 1].max()])
sepal_width['good'] = fuzz.trimf(sepal_width.universe, [sw_mid, x[:, 1].max(), x[:, 1].max()])
#petal_length.automf(3)
pl_mid = x[:, 1].min() + ((x[:, 2].max() - x[:, 2].min())/2)
petal_length['poor'] = fuzz.trimf(petal_length.universe, [x[:, 2].min(), x[:, 2].min(), pl_mid])
petal_length['average'] = fuzz.trimf(petal_length.universe, [x[:, 2].min(), pl_mid, x[:, 2].max()])
petal_length['good'] = fuzz.trimf(petal_length.universe, [pl_mid, x[:, 2].max(), x[:, 2].max()])
petal_width.automf(3)

y_class['Iris-setosa'] = fuzz.trimf(y_class.universe, [0, 0, 1])
y_class['Iris-versicolor'] = fuzz.trimf(y_class.universe, [0, 1, 2])
y_class['Iris-virginica'] = fuzz.trimf(y_class.universe, [1, 2, 2])

In [7]:
rule1 = ctrl.Rule(sepal_length["average"] & sepal_width["average"] & petal_length["poor"] & petal_width["poor"], y_class['Iris-setosa'])
rule2 = ctrl.Rule(sepal_length["average"] & sepal_width["average"] & petal_length["average"] & petal_width["average"], y_class['Iris-versicolor'])
rule3 = ctrl.Rule(sepal_length["good"] & sepal_width["average"] & petal_length["good"] & petal_width["good"], y_class['Iris-virginica'])

In [8]:
iris_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
iris_system = ctrl.ControlSystemSimulation(iris_ctrl)

In [ ]:
# оцінка функції придатності
def evaluate_system(params):
 # задання параметрів нечіткої системи
 param_erythema = params[0] * 0.1 
 param_scaling = params[1] * 0.1